In [68]:
from Crawling_Dataset import Crawling_Dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import cv2 
import os
import numpy as np
from torchvision import transforms
import torch.utils.data as data
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import math

In [14]:
test_path = '/opt/ml/data/celeb_30/cut_test'
train_path = '/opt/ml/data/celeb_30/cut_train'
BATCH_SIZE = 2

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [ ]:
def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    is_fp = np.logical_and(predict_issame, np.logical_not(actual_issame))
    is_fn = np.logical_and(np.logical_not(predict_issame), actual_issame)

    tpr = 0 if (tp+fn==0) else float(tp) / float(tp+fn)
    fpr = 0 if (fp+tn==0) else float(fp) / float(fp+tn)
    acc = float(tp+tn)/dist.size
    return tpr, fpr, acc, is_fp, is_fn

In [74]:
transform = transforms.Compose([
        np.float32,
        transforms.ToTensor(),  # range [0, 255] -> [0.0, 1.0]
        fixed_image_standardization
    ])

In [70]:
dataset = Crawling_Dataset(Enrolled_FILE_PATH=train_path, TEST_FILE_PATH=test_path, transforms=transform)
embedding_loader =data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)

In [71]:
model = InceptionResnetV1(
    classify=False,
    pretrained='vggface2'
).to(device)

In [81]:
model.eval()
with torch.no_grad():
    for enrolled_image, enrolled_label, positive_blocks, negative_blocks in embedding_loader :
        print(enrolled_label)
        for batch in range(BATCH_SIZE) : 
            print(batch)
            sub_enrolled_image = enrolled_image[batch]
            sub_positive_blocks = positive_blocks[batch]
            sub_negative_blocks = negative_blocks[batch]
            
            sub_enrolled_image = sub_enrolled_image.to(device)
            sub_positive_blocks = sub_positive_blocks.to(device)
            sub_negative_blocks = sub_negative_blocks.to(device)

            sub_enrolled_embedding = model(sub_enrolled_image)
            sub_positive_blocks_embedding = model(sub_positive_blocks)
            sub_negative_blocks_embedding = model(sub_negative_blocks)
            sub_enrolled_embedding = sub_enrolled_embedding.to('cpu').numpy()
            sub_positive_blocks_embedding = sub_positive_blocks_embedding.to('cpu').numpy()
            sub_negative_blocks_embedding = sub_negative_blocks_embedding.to('cpu').numpy()
            print(sub_enrolled_embedding.shape)
            print(sub_positive_blocks_embedding.shape)
            print(sub_negative_blocks_embedding.shape)

            p_dis = distance(sub_enrolled_embedding, sub_positive_blocks_embedding)
            n_dis = distance(sub_enrolled_embedding, sub_negative_blocks_embedding)
            print('positive distance : ',p_dis,sep = '\n')
            print('negative distance : ',n_dis, sep = '\n')

            thresholds = np.arange(0, 4, 0.01)
            
        break

('SongJoongki', 'Jojeongseok')
0
(1, 512)
(10, 512)
(10, 512)
positive distance : 
[0.42599598 0.34496748 0.52342516 0.27476713 0.4425782  0.5255498
 0.41805807 0.35854483 0.2805431  0.5122041 ]
negative distance : 
[1.034279   1.4414062  0.6952398  0.84802854 1.1006616  1.0702555
 0.6236588  1.071336   1.0543851  0.8297361 ]
1
(1, 512)
(10, 512)
(10, 512)
positive distance : 
[0.33855894 0.32320276 0.25040722 0.48597103 0.32181776 0.39690956
 0.2592365  0.20619507 0.39997667 0.4392305 ]
negative distance : 
[0.67458963 1.0547748  0.7404216  0.559502   0.60595524 0.7801169
 0.80284184 0.9813439  0.8526923  1.2568748 ]


In [50]:
def distance(embeddings1, embeddings2, distance_metric=0):
    if distance_metric==0:
        # Euclidian distance
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),1)
    elif distance_metric==1:
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi
    else:
        raise 'Undefined distance metric %d' % distance_metric

    return dist